In [15]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import kagglehub
import math
import time
import gc

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

In [3]:
print("Downloading dataset...")
path = kagglehub.dataset_download("bhaveshmittal/melanoma-cancer-dataset")
print("Path to dataset files:", path)

100%|██████████| 79.4M/79.4M [00:04<00:00, 18.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/bhaveshmittal/melanoma-cancer-dataset/versions/1


In [4]:
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
BATCH_SIZE = 32
INPUT_SHAPE = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.4
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.4
)

# Load Data
train_generator = train_datagen.flow_from_directory(
    path + '/train',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=SEED
)

validation_generator = train_datagen.flow_from_directory(
    path + '/train',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=SEED
)

Found 7128 images belonging to 2 classes.
Found 4751 images belonging to 2 classes.


In [6]:
def evaluate_model(
    filters1, filters2, dropout1, dropout2,
    learning_rate, activation, last_activation,
    keep_model=False
):
    model = models.Sequential([
        layers.Conv2D(filters=filters1, kernel_size=(3, 3), activation=activation,
                      input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(filters=filters2, kernel_size=(3, 3), activation=activation),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(filters=128, kernel_size=(3, 3), activation=activation),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dense(units=128, activation=activation),
        layers.Dropout(dropout1),
        layers.Dense(units=64, activation=activation),
        layers.Dropout(dropout2),
        layers.Dense(units=1, activation=last_activation)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=4,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE,
        verbose=0
    )

    val_acc = history.history["val_accuracy"][-1]

    # ===== MEMORY SAFE CLEANUP =====
    if not keep_model:
        tf.keras.backend.clear_session()
        del model
        del history
        del optimizer
        return val_acc, None, None
    # ===============================

    return val_acc, model, history

In [7]:
search_space = {}
search_space['filters1'] = [32, 64, 128]
search_space['filters2'] = [32,64,128]
search_space['dropout1'] = [0.1, 0.3, 0.5, 0.7]
search_space['dropout2'] = [0.1, 0.3, 0.5, 0.7]
search_space['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
search_space['activation'] = ['relu', 'elu', 'gelu']
search_space['last_activation'] = ['sigmoid']

In [8]:
def _immediate_neighbours(index, dimension, state):
    neighbours = []
    vals = search_space[dimension]
    new_state_base = list(state)  # Convert tuple to list for modification

    if index == 0:
        neigh_state = new_state_base.copy()
        neigh_state[list(search_space.keys()).index(dimension)] = vals[1]
        neighbours.append(tuple(neigh_state))

        neigh_state = new_state_base.copy()
        neigh_state[list(search_space.keys()).index(dimension)] = vals[2]
        neighbours.append(tuple(neigh_state))
    elif index == len(vals) - 1:
        neigh_state = new_state_base.copy()
        neigh_state[list(search_space.keys()).index(dimension)] = vals[-2]
        neighbours.append(tuple(neigh_state))

        neigh_state = new_state_base.copy()
        neigh_state[list(search_space.keys()).index(dimension)] = vals[-3]
        neighbours.append(tuple(neigh_state))
    else:
        neigh_state = new_state_base.copy()
        neigh_state[list(search_space.keys()).index(dimension)] = vals[index - 1]
        neighbours.append(tuple(neigh_state))

        neigh_state = new_state_base.copy()
        neigh_state[list(search_space.keys()).index(dimension)] = vals[index + 1]
        neighbours.append(tuple(neigh_state))

    return neighbours

def get_neighbours(state):
    neighbours = []
    state_dict = {k: state[i] for i, k in enumerate(search_space.keys())}

    for k in search_space.keys():
        if k in ['activation', 'last_activation']:
            continue
        index = search_space[k].index(state_dict[k])
        new_neighbours = _immediate_neighbours(index, k, state)
        neighbours.extend(new_neighbours)

    # Add a neighbor with random activation and last_activation
    new_state = list(state)
    new_state[5] = np.random.choice(search_space['activation'])
    new_state[6] = np.random.choice(search_space['last_activation'])
    neighbours.append(tuple(new_state))

    return neighbours

In [9]:
def whale_search(max_iter=10, population_size=3, b=1.0):
    print(f"  -> WOA Start (Pop={population_size}, Iter={max_iter}, b={b})")
    keys = list(search_space.keys())
    population = []
    for _ in range(population_size):
        w = (
            random.choice(search_space["filters1"]),
            random.choice(search_space["filters2"]),
            random.choice(search_space["dropout1"]),
            random.choice(search_space["dropout2"]),
            random.choice(search_space["learning_rate"]),
            random.choice(search_space["activation"]),
            random.choice(search_space["last_activation"])
        )
        population.append(w)

    scores = []
    # --- Evaluate Initial Population ---
    for w in population:
        # keep_model=False ensures we don't return the heavy object
        acc, _, _ = evaluate_model(*w, keep_model=False)
        scores.append(acc)
        # CLEANUP
        tf.keras.backend.clear_session()
        gc.collect()

    best_index = int(np.argmax(scores))
    best_state = population[best_index]
    best_score = scores[best_index]

    for t in range(max_iter):
        a = 2 - 2 * (t / max_iter)
        new_population = []

        for i in range(population_size):
            whale = population[i]
            p = random.random()
            # Convert categorical params to indices for math operations
            whale_indices = [
                search_space["filters1"].index(whale[0]),
                search_space["filters2"].index(whale[1]),
                search_space["dropout1"].index(whale[2]),
                search_space["dropout2"].index(whale[3]),
                search_space["learning_rate"].index(whale[4])
            ]

            best_indices = [
                search_space["filters1"].index(best_state[0]),
                search_space["filters2"].index(best_state[1]),
                search_space["dropout1"].index(best_state[2]),
                search_space["dropout2"].index(best_state[3]),
                search_space["learning_rate"].index(best_state[4])
            ]

            updated_indices = []

            # Update numeric parameters (first 5 params)
            for dim in range(len(whale_indices)):
                r1 = random.random()
                r2 = random.random()
                A = 2 * a * r1 - a
                C = 2 * r2

                val = 0

                if p < 0.5:
                    if abs(A) < 1:
                        # Encircling prey
                        D = abs(C * best_indices[dim] - whale_indices[dim])
                        val = best_indices[dim] - A * D
                    else:
                        # Search for prey (exploration)
                        rand_idx = random.randint(0, population_size - 1)
                        rand_whale = population[rand_idx]
                        rand_val_idx = random.randint(0, len(search_space[keys[dim]]) - 1)
                        D = abs(C * rand_val_idx - whale_indices[dim])
                        val = rand_val_idx - A * D
                else:
                    # Spiral updating
                    D_prime = abs(best_indices[dim] - whale_indices[dim])
                    l = random.uniform(-1, 1)
                    val = D_prime * math.exp(b * l) * math.cos(2 * math.pi * l) + best_indices[dim]

                val = int(round(val))
                val = max(0, min(val, len(search_space[keys[dim]]) - 1))
                updated_indices.append(val)

            new_state = (
                search_space["filters1"][updated_indices[0]],
                search_space["filters2"][updated_indices[1]],
                search_space["dropout1"][updated_indices[2]],
                search_space["dropout2"][updated_indices[3]],
                search_space["learning_rate"][updated_indices[4]],
                random.choice(search_space["activation"]),      # Randomly mutated
                random.choice(search_space["last_activation"])  # Randomly mutated
            )
            new_population.append(new_state)

        # --- Evaluate New Positions ---
        new_scores = []
        for w in new_population:
            acc, _, _ = evaluate_model(*w, keep_model=False)
            new_scores.append(acc)

            # CLEANUP
            tf.keras.backend.clear_session()
            gc.collect()

        # Check for improvement in this batch
        iteration_best_index = int(np.argmax(new_scores))
        iteration_best_score = new_scores[iteration_best_index]
        iteration_best_state = new_population[iteration_best_index]

        if iteration_best_score > best_score:
            best_score = iteration_best_score
            best_state = iteration_best_state

        population = new_population

    print(f"  -> WOA Finished. Best Score: {best_score:.4f}")

    return best_state, best_score

In [12]:
start_time = time.time()
best_state, best_score = whale_search()
end_time = time.time()
print(f"Total time taken: {end_time - start_time:.2f} seconds")
print(f"Best State: {best_state}")
print(f"Best Score: {best_score:.4f}")

  -> WOA Start (Pop=3, Iter=10, b=1.0)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building you

  -> WOA Finished. Best Score: 0.8161
Total time taken: 2185.08 seconds
Best State: (32, 128, 0.7, 0.1, 0.0001, 'elu', 'sigmoid')
Best Score: 0.8161


In [13]:
test_generator = test_datagen.flow_from_directory(
    path + '/test',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=False,
    seed=SEED
)

Found 1200 images belonging to 2 classes.


In [16]:
filters1, filters2, dropout1, dropout2, learning_rate, activation, last_activation = best_state

optimized_model = models.Sequential([
    layers.Conv2D(filters=filters1, kernel_size=(3, 3), activation=activation, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(filters=filters2, kernel_size=(3, 3), activation=activation),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(filters=128, kernel_size=(3, 3), activation=activation),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.Dense(units=128, activation=activation),
    layers.Dropout(dropout1),
    layers.Dense(units=64, activation=activation),
    layers.Dropout(dropout2),
    layers.Dense(units=1, activation=last_activation)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
optimized_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

history = optimized_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping, reduce_lr]
)

test_loss, test_acc = optimized_model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

optimized_model.save('WOA_optimized_melanoma_model.keras')
print("Final model saved as WOA_optimized_melanoma_model.keras")

Epoch 1/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 28s 106ms/step - accuracy: 0.6596 - loss: 0.6062 - val_accuracy: 0.6634 - val_loss: 0.6635 - learning_rate: 1.0000e-04
Epoch 2/20
  1/222 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5938 - loss: 0.7916

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - accuracy: 0.5938 - loss: 0.7916 - val_accuracy: 0.7238 - val_loss: 0.5158 - learning_rate: 1.0000e-04
Epoch 3/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 21s 95ms/step - accuracy: 0.8126 - loss: 0.4153 - val_accuracy: 0.8081 - val_loss: 0.4461 - learning_rate: 1.0000e-04
Epoch 4/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.9062 - loss: 0.3269 - val_accuracy: 0.8051 - val_loss: 0.4575 - learning_rate: 1.0000e-04
Epoch 5/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 20s 91ms/step - accuracy: 0.8155 - loss: 0.3958 - val_accuracy: 0.7720 - val_loss: 0.4619 - learning_rate: 1.0000e-04
Epoch 6/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.8750 - loss: 0.3203 - val_accuracy: 0.7844 - val_loss: 0.4434 - learning_rate: 1.0000e-04
Epoch 7/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 21s 96ms/step - accuracy: 0.8372 - loss: 0.3605 - val_accuracy: 0.7804 - val_loss: 0.4486 - learning_rate: 1.0000e-04
Epoch 8/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.8634 - loss: 0.2999
Test accuracy: 0.8075000047683716
Final model saved as WOA_optimized_melanoma_model.keras
